<h1> User Engagement Analysis </h2>

<h2> Imporing data and packages </h2>

In [1]:
import pandas as pd
import numpy as np
import os
import warnings
from pandasql import sqldf
import matplotlib.pyplot as plt
import seaborn as sns
import sys
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans

In [2]:
# Suppressing warning messages

warnings.filterwarnings('ignore')

In [3]:
# setting path to file and folders

parent_dir = os.path.abspath(os.path.join(os.getcwd(), os.pardir)) # setting path to parent directory
scripts_dir = os.path.join(parent_dir,"scripts") # setting path to scripts directory
data_path = os.path.join(parent_dir,"data","teleco_user_exp_data.csv") # setting path to data directory

sys.path.insert(1, scripts_dir)

In [5]:
# Importing scripts

from data_cleaning_functions import DataCleaner as Cleaner 
from data_summarizing_functions import DataSummarizer as Sumar

cleaner = Cleaner()
sumar = Sumar()

In [6]:
# Importing the users data frame with experience and engagement score

user_df= pd.read_csv(data_path)

In [7]:
user_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 106856 entries, 0 to 106855
Data columns (total 22 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   MSISDN/Number      106856 non-null  float64
 1   xDr_session_count  106856 non-null  int64  
 2   session_dur        106856 non-null  float64
 3   Handset            106856 non-null  object 
 4   Total_DL           106856 non-null  float64
 5   Total_UL           106856 non-null  float64
 6   total_vol          106856 non-null  float64
 7   RTT total          106856 non-null  float64
 8   TP total           106856 non-null  float64
 9   TCP total          106856 non-null  float64
 10  youtube            106856 non-null  float64
 11  netflix            106856 non-null  float64
 12  google             106856 non-null  float64
 13  gaming             106856 non-null  float64
 14  email              106856 non-null  float64
 15  social             106856 non-null  float64
 16  ot

In [ ]:
#